In [1]:
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv()

True

## Using Tesla to create Vader sentiment

In [2]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Reading the News API key enviroment variable
api_key = os.getenv("news_api")


In [4]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)


In [5]:
# Fetch all the news about Tesla
tesla_headlines = newsapi.get_everything(
    q="tesla",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Tesla: {tesla_headlines['totalResults']}")

# Show sample article
tesla_headlines["articles"][0]

Total articles about Tesla: 7153


{'source': {'id': 'engadget', 'name': 'Engadget'},
 'author': 'https://www.engadget.com/about/editors/jon-fingas',
 'title': "Elon Musk says Tesla Model S Plaid+ is 'canceled'",
 'description': "Elon Musk claims the Tesla Model S Plaid+ is 'canceled' as the regular Plaid model is good enough..",
 'url': 'https://www.engadget.com/elon-musk-says-tesla-model-s-plaid-plus-canceled-211804141.html',
 'urlToImage': 'https://s.yimg.com/os/creatr-uploaded-images/2021-03/a53fc4a0-8d72-11eb-bffa-fac9a7f8b050',
 'publishedAt': '2021-06-06T21:18:04Z',
 'content': 'You might have to forego dreams of driving Tesla\'s supercar-level Model S Plaid+ sedan. Elon Musk told Twitter followers that the Plaid+ trim is "canceled" as the imminent, lower-priced \'regular\' Pla… [+1150 chars]'}

In [6]:
# Create the Tesla sentiment scores DataFrame
tesla_sentiments = []

for article in tesla_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        tesla_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
tesla_df = pd.DataFrame(tesla_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
tesla_df = tesla_df[cols]

tesla_df.head()


,date,text,compound,positive,negative,neutral
0,2021-06-06,You might have to forego dreams of driving Tes...,0.4019,0.08,0.000,0.920
1,2021-06-07,"Jerome Guillen, a critical executive at Tesla ...",-0.3182,0.00,0.063,0.937
2,2021-06-11,A lucky 25 customers received Tesla’s redesign...,0.4215,0.08,0.000,0.920
3,2021-06-14,"Redwood Materials, the battery recycling start...",0.0000,0.00,0.000,1.000
4,2021-06-22,Tesla CEO Elon Musk has been teasing a neural ...,-0.0772,0.00,0.036,0.964


In [7]:
tesla_df.describe()


,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.124687,0.049600,0.022700,0.927710
std,0.372240,0.058857,0.044802,0.067081
min,-0.796400,0.000000,0.000000,0.787000
25%,0.000000,0.000000,0.000000,0.871750
50%,0.000000,0.000000,0.000000,0.935000
75%,0.421500,0.088750,0.008000,1.000000
max,0.831400,0.205000,0.213000,1.000000
